In [ ]:
import pandas as pd

df = pd.read_csv("https://data.cityofchicago.org/resource/b8xg-w8bx.csv")
df.head()

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,extras,trip_total,payment_type,company,pickup_centroid_latitude,pickup_centroid_longitude,pickup_centroid_location,dropoff_centroid_latitude,dropoff_centroid_longitude,dropoff_centroid_location
0,fe5cef439c882c0fbc46fb6bad0617b8ecd48499,75ff7e74c62fa9fb70671ee63e3c87153b3037fe0c7490...,2025-11-01T00:00:00.000,2025-11-01T00:00:00.000,595.0,4.12,NaN,NaN,22.0,16.0,...,0.0,13.00,Mobile,Flash Cab,41.922761,-87.699155,POINT (-87.6991553432 41.9227606205),41.953582,-87.723452,POINT (-87.7234523905 41.9535821253)
1,fe0d164107a035984123dd54c9d786d442de0b44,d6e1a9e103336c396201abe9ceb00795fcd41e14ccbf54...,2025-11-01T00:00:00.000,2025-11-01T00:15:00.000,1015.0,4.77,NaN,NaN,32.0,24.0,...,0.0,17.67,Mobile,Flash Cab,41.878866,-87.625192,POINT (-87.6251921424 41.8788655841),41.901207,-87.676356,POINT (-87.6763559892 41.9012069941)
2,fc8b83ceee70e17e2b101b09535ccf927b5ed0e8,2059659f66366de2e2b2e0ea721c8a4f18d61f51547a04...,2025-11-01T00:00:00.000,2025-11-01T00:30:00.000,1622.0,7.57,NaN,NaN,28.0,6.0,...,0.0,21.45,Mobile,Sun Taxi,41.874005,-87.663518,POINT (-87.6635175498 41.874005383),41.944227,-87.655998,POINT (-87.6559981815 41.9442266014)
3,f99fa7b83dc11f8265cba290ad94558a87ba5d50,c6c0742ffcad334c2dec7af4e40a6b75e0e641f8efc617...,2025-11-01T00:00:00.000,2025-11-01T00:15:00.000,1354.0,16.61,1.703198e+10,1.703184e+10,76.0,32.0,...,4.0,54.90,Credit Card,Chicago Independents,41.979071,-87.903040,POINT (-87.9030396611 41.9790708201),41.880994,-87.632746,POINT (-87.6327464887 41.8809944707)
4,f8ef05929b72ebddba858cdf069d2ab1c2d1fc31,2d5029b701200a2ab66bd5071f743efe20a3036597d2b3...,2025-11-01T00:00:00.000,2025-11-01T00:15:00.000,1432.0,15.43,NaN,NaN,76.0,24.0,...,4.0,46.25,Credit Card,Sun Taxi,41.980264,-87.913625,POINT (-87.913624596 41.9802643146),41.901207,-87.676356,POINT (-87.6763559892 41.9012069941)


In [ ]:
df['trip_start_timestamp'] = pd.to_datetime(df['trip_start_timestamp'])
df['trip_end_timestamp'] = pd.to_datetime(df['trip_end_timestamp'])

df['trip_duration_min'] = (df['trip_end_timestamp'] - df['trip_start_timestamp']).dt.total_seconds() / 60
df['hour'] = df['trip_start_timestamp'].dt.hour
df['day'] = df['trip_start_timestamp'].dt.day
df['month'] = df['trip_start_timestamp'].dt.month
df['year'] = df['trip_start_timestamp'].dt.year

df = df.dropna(subset=['trip_miles', 'fare'])

In [ ]:
fact_table = df[['trip_duration_min', 'trip_miles', 'fare', 'tips', 'trip_total']]

In [ ]:
fact_table.to_csv("fact_taxi_trips.csv", index=False)

In [ ]:
fact_table.head()

,trip_duration_min,trip_miles,fare,tips,trip_total
0,0.0,4.12,12.50,0.00,13.00
1,15.0,4.77,17.17,0.00,17.67
2,30.0,7.57,20.95,0.00,21.45
3,15.0,16.61,41.25,9.15,54.90
4,15.0,15.43,38.75,3.00,46.25


In [ ]:
import os
os.listdir()

['.config', 'fact_taxi_trips.csv', 'sample_data']

In [4]:
import sqlite3
import pandas as pd

df = pd.read_csv("https://data.cityofchicago.org/resource/b8xg-w8bx.csv")
df.head()

df['trip_start_timestamp'] = pd.to_datetime(df['trip_start_timestamp'])
df['trip_end_timestamp'] = pd.to_datetime(df['trip_end_timestamp'])

df['trip_duration_min'] = (
    df['trip_end_timestamp'] - df['trip_start_timestamp']
).dt.total_seconds() / 60

df['hour'] = df['trip_start_timestamp'].dt.hour
df['day'] = df['trip_start_timestamp'].dt.day
df['month'] = df['trip_start_timestamp'].dt.month
df['year'] = df['trip_start_timestamp'].dt.year

df = df.dropna(subset=['trip_miles', 'fare'])

In [5]:
slice_january = df[df['month'] == 1]
slice_january[['trip_miles', 'fare']].head()

,trip_miles,fare


In [6]:
dice_night_cc = df[
    (df['payment_type'] == 'Credit Card') &
    ((df['hour'] >= 22) | (df['hour'] <= 2))
]

dice_night_cc[['hour', 'payment_type', 'fare']].head()

,hour,payment_type,fare
3,0,Credit Card,41.25
4,0,Credit Card,38.75
5,0,Credit Card,3.25
9,0,Credit Card,20.00
19,0,Credit Card,40.00


In [7]:
rollup_monthly = df.groupby('month')['fare'].sum().reset_index()
rollup_monthly

,month,fare
0,10,18105.17
1,11,1760.85


In [8]:
drilldown = df.groupby(['year', 'month', 'day'])['fare'].sum().reset_index()
drilldown.head()

,year,month,day,fare
0,2025,10,31,18105.17
1,2025,11,1,1760.85


In [9]:
pivot_table = pd.pivot_table(
    df,
    values='fare',
    index='month',
    columns='payment_type',
    aggfunc='sum'
)

pivot_table

payment_type,Cash,Credit Card,Dispute,Mobile,No Charge,Prcard,Unknown
month,,,,,,,
10,3186.04,7209.57,21.0,6950.06,25.0,628.5,85.00
11,268.76,687.50,NaN,713.84,5.0,56.5,29.25
